## Email w/ Python

In [ ]:
import smtplib
import getpass

In [2]:
smtp_obj = smtplib.SMTP('smtp.gmail.com', 587)

smtp_obj.ehlo()

(250,
 b'smtp.gmail.com at your service, [73.244.171.6]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [3]:
smtp_obj.starttls()

(220, b'2.0.0 Ready to start TLS')

### GMail API Test
### ======================================================

In [9]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import base64
import os.path
import pickle
from datetime import datetime

Packages to install in your env :

```bash
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

GCP Service account:  ```moneypenny@python-resource-mgt.iam.gserviceaccount.com```

### OAuth Scopes:

* Gmail:
    * ```https://www.googleapis.com/auth/gmail.modify```
    * ```https://www.googleapis.com/auth/gmail.labels```

* Sheets:
    * ```https://www.googleapis.com/auth/spreadsheets```

* Drive:
    * ```https://www.googleapis.com/auth/drive.file```


In [15]:
!ls -l /mnt/d/GCP/

total 4
-rwxrwxrwx 1 tfadmin tfadmin  407 Dec  8 18:13 moneypenny_oauth_id.json
-rwxrwxrwx 1 tfadmin tfadmin 2371 Dec  8 15:41 python-resource-mgt-0ac59ab81869.json


In [18]:
def create_gmail_service():
    """
    Create a Gmail service using OAuth 2.0 credentials.
    Handles token creation and refresh, with manual browser authorization.
    """
    creds = None
    # Token file stores the user's access and refresh tokens
    TOKEN_FILE = 'token.pickle'
    # If you have a credentials.json file from Google Cloud Console
    CLIENT_SECRETS_FILE = '/mnt/d/GCP/moneypenny_oauth_id.json'
    
    # The scope we need for Gmail API
    SCOPES = ['https://www.googleapis.com/auth/gmail.modify',
              'https://www.googleapis.com/auth/gmail.labels']

    # Load existing tokens if they exist
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials available, let the user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, 
                SCOPES,
                redirect_uri='http://localhost:8080'
            )
            
            # Get the authorization URL
            auth_url, _ = flow.authorization_url(
                access_type='offline',
                include_granted_scopes='true'
            )
            
            print('Please go to this URL and authorize the application:')
            print(auth_url)
            print('\nAfter authorization, you\'ll be redirected to localhost.')
            print('Copy the entire URL from your browser and paste it here: ')
            redirect_response = input()
            
            flow.fetch_token(authorization_response=redirect_response)
            creds = flow.credentials
            
        # Save the credentials for the next run
        with open(TOKEN_FILE, 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

def test_gmail_functionality(gmail_service, user_id='me'):
    """
    Test various Gmail API functionalities
    
    Args:
        gmail_service: Authenticated Gmail service object
        user_id: User's email address. Special value 'me' for authenticated user.
    """
    # Test 1: List labels
    def list_labels():
        results = gmail_service.users().labels().list(userId=user_id).execute()
        labels = results.get('labels', [])
        print("Labels:")
        for label in labels:
            print(f"- {label['name']} ({label['id']})")

    # Test 2: Create a new label
    def create_label(label_name="Test_Label"):
        label_object = {
            'name': label_name,
            'labelListVisibility': 'labelShow',
            'messageListVisibility': 'show'
        }
        try:
            created_label = gmail_service.users().labels().create(
                userId=user_id,
                body=label_object
            ).execute()
            print(f"Created label: {created_label['name']}")
            return created_label
        except Exception as e:
            print(f"Error creating label: {e}")

    # Test 3: Send an email
    def send_email(to, subject, message_text):
        message = MIMEText(message_text)
        message['to'] = to
        message['subject'] = subject
        
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
        try:
            sent_message = gmail_service.users().messages().send(
                userId=user_id,
                body={'raw': raw_message}
            ).execute()
            print(f"Message sent. Message ID: {sent_message['id']}")
            return sent_message
        except Exception as e:
            print(f"Error sending message: {e}")

    # Test 4: List recent messages
    def list_recent_messages(max_results=5):
        results = gmail_service.users().messages().list(
            userId=user_id,
            maxResults=max_results
        ).execute()
        messages = results.get('messages', [])
        print(f"\nRecent {max_results} messages:")
        for message in messages:
            msg = gmail_service.users().messages().get(
                userId=user_id,
                id=message['id']
            ).execute()
            print(f"- Subject: {next((header['value'] for header in msg['payload']['headers'] if header['name'] == 'Subject'), 'No Subject')}")

    # Run tests
    print("Starting Gmail API tests...")
    list_labels()
    created_label = create_label("Test_Label_" + datetime.now().strftime("%Y%m%d_%H%M%S"))
    send_email("your-email@example.com", "Test Email", "This is a test email sent via Gmail API")
    list_recent_messages()

def main():
    # Create Gmail service using OAuth
    gmail_service = create_gmail_service()
    
    # Run the tests
    test_gmail_functionality(gmail_service)

if __name__ == "__main__":
    main()

Please go to this URL and authorize the application:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=336348628770-ql8ed1mok8mq938h00n0sl75pqud9n5j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.labels&state=P7IoPDT8PLV1iXfd8OnhAs3THsJmro&access_type=offline&include_granted_scopes=true

After authorization, you'll be redirected to localhost.
Copy the entire URL from your browser and paste it here: 


ValueError: Please supply either code or authorization_response parameters.